# Predicting "Opioid Prescribers" in Emergency Medicine: New Mexico Focus

**The overarching aim of this project is to predict which Doctors will prescribe higher than average amounts of opiods and, thus, be termed an "Opiod Prescriber"


**The goals are to:**
- Wrangle the data
- Predict opiod prescribers in the USA
- Find which predictors are most significant
- Analyze and compare possible trends in opiod prescription within subfields in the USA and New Mexico, USA

**Dataset:**
The open source data for this project was acquired (and can be downloaded) from: https://www.kaggle.com/apryor6/us-opiate-prescriptions
The data is made up of three .csv files "opiods.csv", "overdoses.csv", and "prescriber-info.csv"

This dataset contains summaries of prescription records for 250 common opioid and non-opioid drugs written by 25,000 unique licensed medical professionals in 2014 in the United States for citizens covered under Class D Medicare as well as some metadata about the doctors themselves. This is a small subset of data that was sourced from cms.gov. The full dataset contains almost 24 million prescription instances in long format. Dr. Alan ("AJ") Pryor (who posted this dataset on Kaggle) has cleaned and compiled this data here in a format with 1 row per prescriber and limited the approximately 1 million total unique prescribers down to 25,000 to keep it manageable. For instructions on getting the full data, plese refer to the instructions listed by Dr. Pryor in the kaggle link above.

## **PART 1:**

## **Setup, Wrangling, and EDA of non-drug features**

### A. Import packages and get the data

In [920]:
#for data analysis and wrangling
import pandas as pd
import numpy as np
import random as rnd
import re

#for data visualizations
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

#for machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris

In [921]:
prescriber = pd.read_csv('/Users/rmtaylor/ML/Projects/Opioid_ER/prescriber-info.csv')
opioids = pd.read_csv('/Users/rmtaylor/ML/Projects/Opioid_ER/opioids.csv')
overdose = pd.read_csv('/Users/rmtaylor/ML/Projects/Opioid_ER/overdoses.csv')

### B. First look at the PRESCRIBER DATASET first

#### 1. Prescriber Dataset

In [922]:
prescriber.head()

,NPI,Gender,State,Credentials,Specialty,ABILIFY,ACETAMINOPHEN.CODEINE,ACYCLOVIR,ADVAIR.DISKUS,AGGRENOX,...,VERAPAMIL.ER,VESICARE,VOLTAREN,VYTORIN,WARFARIN.SODIUM,XARELTO,ZETIA,ZIPRASIDONE.HCL,ZOLPIDEM.TARTRATE,Opioid.Prescriber
0,1710982582,M,TX,DDS,Dentist,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,1245278100,F,AL,MD,General Surgery,0,0,0,0,0,...,0,0,0,0,0,0,0,0,35,1
2,1427182161,F,NY,M.D.,General Practice,0,0,0,0,0,...,0,0,0,0,0,0,0,0,25,0
3,1669567541,M,AZ,MD,Internal Medicine,0,43,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,1679650949,M,NV,M.D.,Hematology/Oncology,0,0,0,0,0,...,0,0,0,0,17,28,0,0,0,1


In [923]:
prescriber.columns

Index(['NPI', 'Gender', 'State', 'Credentials', 'Specialty', 'ABILIFY',
       'ACETAMINOPHEN.CODEINE', 'ACYCLOVIR', 'ADVAIR.DISKUS', 'AGGRENOX',
       ...
       'VERAPAMIL.ER', 'VESICARE', 'VOLTAREN', 'VYTORIN', 'WARFARIN.SODIUM',
       'XARELTO', 'ZETIA', 'ZIPRASIDONE.HCL', 'ZOLPIDEM.TARTRATE',
       'Opioid.Prescriber'],
      dtype='object', length=256)

In [924]:
prescriber.shape

(25000, 256)

We can see that that:
- There is a prescriber #, gender, state, credentials, and specialty followed by a list of drugs prescribed

**Let's see what the different specialties in the Specialty feature are...**

In [925]:
prescriber["Specialty"].value_counts().sort_values(ascending = False)

Internal Medicine                                                 3194
Family Practice                                                   2975
Dentist                                                           2800
Nurse Practitioner                                                2512
Physician Assistant                                               1839
Emergency Medicine                                                1087
Psychiatry                                                         691
Cardiology                                                         688
Obstetrics/Gynecology                                              615
Orthopedic Surgery                                                 575
Optometry                                                          571
Student in an Organized Health Care Education/Training Program     547
Ophthalmology                                                      519
General Surgery                                                    487
Gastro

**We can see that there are 109 different specialties listed (length)**

In [926]:
len(prescriber["Specialty"].value_counts().sort_values(ascending = False))

109

**Let's look at the credentials feature now...**

In [927]:
prescriber["Credentials"].value_counts().sort_values(ascending = False)

MD                      7034
M.D.                    6772
DDS                     1145
D.O.                     866
PA-C                     845
D.D.S.                   717
DO                       549
NP                       469
DMD                      449
PA                       437
O.D.                     353
FNP                      262
ARNP                     256
DPM                      247
M.D                      243
D.M.D.                   234
OD                       175
P.A.                     168
CRNP                     150
APRN                     134
N.P.                     105
PAC                       98
D.P.M.                    98
CNP                       86
D.D.S                     85
NP-C                      84
FNP-C                     70
APN                       66
FNP-BC                    56
P.A.-C                    52
D.M.D                     48
D.O                       47
M. D.                     45
RPA-C                     41
ANP           

##### From looking at the credentials, we see that:
- We will need to clean the credentials by:

    **-We will need to use Regex to make all of the same credentials have the same format (i.e. no periods or spaces)**
    - removing periods, commas, and dashes in all credentials
    - for all listings with multiple degrees in a single row, we will create new features called "Medical", "Doctorate", "Master", "Bachelor", "Associate", and "License" 
    - We will then look at the credential list again to make sure we have cleaned up all info.

**Let's start by removing periods and then replacing commas with a space**

In [928]:
prescriber = prescriber.replace({'Credentials': r'\.'}, {'Credentials': ''}, regex=True)
prescriber = prescriber.replace({'Credentials': r','}, {'Credentials': ' '}, regex=True)

In [929]:
prescriber["Credentials"].value_counts().sort_values(ascending = False)

MD                      14060
DDS                      1947
DO                       1462
PA-C                      923
DMD                       731
PA                        614
NP                        581
OD                        532
DPM                       354
FNP                       289
ARNP                      273
CRNP                      157
APRN                      139
PAC                       104
CNP                        92
MD  PHD                    87
NP-C                       85
FNP-C                      73
APN                        70
FNP-BC                     56
M D                        51
ANP                        47
MD  MPH                    43
RPA-C                      41
CFNP                       29
MD PHD                     28
MBBS                       26
MD                         25
PHARMD                     24
NURSE PRACTITIONER         23
APNP                       21
DDS  MS                    21
ANP-BC                     20
PHYSICIAN 

**Let's take care of a few special cases now before we split the column...**

**I'm formating the credentials so that I can split the column**

In [930]:
prescriber = prescriber.replace({'Credentials': r'M D'}, {'Credentials': 'MD'}, regex=True)
prescriber = prescriber.replace({'Credentials': r'D O'}, {'Credentials': 'DO'}, regex=True)
prescriber = prescriber.replace({'Credentials': r'O D'}, {'Credentials': 'OD'}, regex=True)
prescriber = prescriber.replace({'Credentials': r'D D S'}, {'Credentials': 'DDS'}, regex=True)
prescriber = prescriber.replace({'Credentials': r'P A'}, {'Credentials': 'PA'}, regex=True)
prescriber = prescriber.replace({'Credentials': r'MD>'}, {'Credentials': 'MD'}, regex=True)
prescriber = prescriber.replace({'Credentials': r'D D S M D'}, {'Credentials': 'DDS MD'}, regex=True)
prescriber = prescriber.replace({'Credentials': r'D D O M S'}, {'Credentials': 'DDO MS'}, regex=True)
prescriber = prescriber.replace({'Credentials': r'C FNP'}, {'Credentials': 'NP'}, regex=True)
prescriber = prescriber.replace({'Credentials': r'M D P C'}, {'Credentials': 'MD'}, regex=True)
prescriber = prescriber.replace({'Credentials': r'M D MPH'}, {'Credentials': 'MD MPH'}, regex=True)
prescriber = prescriber.replace({'Credentials': r'MD ; MB BS'}, {'Credentials': 'MD MB MS'}, regex=True)
prescriber = prescriber.replace({'Credentials': r'AT C CSCS'}, {'Credentials': 'ATC CSCS'}, regex=True)
prescriber = prescriber.replace({'Credentials': r'MS;BS'}, {'Credentials': 'MS BS'}, regex=True)
prescriber = prescriber.replace({'Credentials': r'D O P C'}, {'Credentials': 'DO'}, regex=True)
prescriber = prescriber.replace({'Credentials': r'ACNP/FNP'}, {'Credentials': 'NP'}, regex=True)
prescriber = prescriber.replace({'Credentials': r'M D F A A D'}, {'Credentials': 'MD FAAD'}, regex=True)
prescriber = prescriber.replace({'Credentials': r'MD/'}, {'Credentials': 'MD'}, regex=True)
prescriber = prescriber.replace({'Credentials': r'PH D PA-C'}, {'Credentials': 'PA PHD'}, regex=True)
prescriber = prescriber.replace({'Credentials': r' (DMD)'}, {'Credentials': 'DMD'}, regex=True)
prescriber = prescriber.replace({'Credentials': r'PH D MD'}, {'Credentials': 'MD PHD'}, regex=True)
prescriber = prescriber.replace({'Credentials': r'DMD M S'}, {'Credentials': 'DMD MS'}, regex=True)
prescriber = prescriber.replace({'Credentials': r'D P M'}, {'Credentials': 'DPM'}, regex=True)
prescriber = prescriber.replace({'Credentials': r'MD/PHD'}, {'Credentials': 'MD PHD'}, regex=True)
prescriber = prescriber.replace({'Credentials': r'DD S'}, {'Credentials': 'DDS'}, regex=True)
prescriber = prescriber.replace({'Credentials': r'MDPHD'}, {'Credentials': 'MD PHD'}, regex=True)
prescriber = prescriber.replace({'Credentials': r'MDPA'}, {'Credentials': 'MD PA'}, regex=True)
prescriber = prescriber.replace({'Credentials': r'R PA-C'}, {'Credentials': 'PA'}, regex=True)
prescriber = prescriber.replace({'Credentials': r'MDFACP'}, {'Credentials': 'MD FACP'}, regex=True)

In [931]:
prescriber["Credentials"].value_counts().sort_values(ascending = False)

MD                      14113
DDS                      1953
DO                       1469
PA-C                      923
DMD                       731
PA                        616
NP                        582
OD                        536
DPM                       356
FNP                       289
ARNP                      273
CRNP                      157
APRN                      139
PAC                       104
CNP                        92
MD  PHD                    87
NP-C                       85
FNP-C                      73
APN                        70
FNP-BC                     56
ANP                        47
MD  MPH                    43
RPA-C                      41
MD PHD                     33
PHARMD                     31
CFNP                       29
MD                         28
MBBS                       26
NURSE PRACTITIONER         23
DDS  MS                    21
APNP                       21
MD PA                      20
ANP-BC                     20
MD MPH    

**Now I'll split the column...**

In [932]:
# new data frame with split value columns 
new = prescriber["Credentials"].str.split(" ", n = 1, expand = True) 
  
# making seperate Cred1 (1st credential) column from new data frame 
prescriber["Cred1"]= new[0] 
  
# making seperate Cred2 column from new data frame 
prescriber["Cred2"]= new[1] 

In [933]:
pd.set_option('display.max_rows', 800)
prescriber["Cred1"].value_counts().sort_values(ascending = False)

MD                14480
DDS                2071
DO                 1492
PA-C                941
DMD                 777
PA                  639
NP                  592
OD                  547
DPM                 362
FNP                 303
ARNP                290
APRN                180
CRNP                161
PAC                 104
RN                  100
CNP                  95
NP-C                 86
APN                  81
FNP-C                74
FNP-BC               62
MSN                  61
ANP                  49
RPA-C                42
MBBS                 34
DNP                  33
PHARMD               32
MS                   31
CFNP                 29
NURSE                25
PHYSICIAN            23
ANP-BC               21
APNP                 21
CNM                  21
CNS                  15
APRN-BC              14
ND                   13
PHD                  13
RPAC                 13
PMHNP                12
ACNP                 12
ACNP-BC              12
MPAS            

In [934]:
len(prescriber["Cred1"].value_counts().sort_values(ascending = False))

205

**We have 205 different credentials or formats for credentials**

Let's now combine degrees that are the same and also fill in nan values with "unknown"

In [935]:
# Nurse Practioner records:
for x in range (0, 6):
    prescriber = prescriber.replace({'Cred1': r'.NP+'}, {'Cred1': 'NP'}, regex=True)
    prescriber = prescriber.replace({'Cred1': r'.NP.+'}, {'Cred1': 'NP'}, regex=True)
    prescriber = prescriber.replace({'Cred1': r'NP+'}, {'Cred1': 'NP'}, regex=True)
    prescriber = prescriber.replace({'Cred1': r'NP.+'}, {'Cred1': 'NP'}, regex=True)
    prescriber = prescriber.replace({'Cred1': r'NP'}, {'Cred1': 'NP'}, regex=True)
    prescriber = prescriber.replace({'Cred1': r'NP.'}, {'Cred1': 'NP'}, regex=True)
    
    prescriber = prescriber.replace({'Cred1': r'.RN+'}, {'Cred1': 'NP'}, regex=True)
    prescriber = prescriber.replace({'Cred1': r'RN+'}, {'Cred1': 'NP'}, regex=True)
    
    prescriber = prescriber.replace({'Cred1': r'.PN+'}, {'Cred1': 'NP'}, regex=True)
    prescriber = prescriber.replace({'Cred1': r'PN+'}, {'Cred1': 'NP'}, regex=True)
    
    prescriber = prescriber.replace({'Cred1': r'NURSE'}, {'Cred1': 'NP'}, regex=True)
    
    prescriber = prescriber.replace({'Cred1': r'.CNS+'}, {'Cred1': 'NP'}, regex=True)
    prescriber = prescriber.replace({'Cred1': r'.CNS.+'}, {'Cred1': 'NP'}, regex=True)
    prescriber = prescriber.replace({'Cred1': r'CNS+'}, {'Cred1': 'NP'}, regex=True)
    prescriber = prescriber.replace({'Cred1': r'CNS.+'}, {'Cred1': 'NP'}, regex=True)
    prescriber = prescriber.replace({'Cred1': r'CNS'}, {'Cred1': 'NP'}, regex=True)
    prescriber = prescriber.replace({'Cred1': r'CNS.'}, {'Cred1': 'NP'}, regex=True)
    
    prescriber = prescriber.replace({'Cred1': r'ANRP'}, {'Cred1': 'NP'}, regex=True)
    prescriber = prescriber.replace({'Cred1': r'CNFP'}, {'Cred1': 'NP'}, regex=True)
    
    
    
    print (x, 'Changes Complete')

0 Changes Complete
1 Changes Complete
2 Changes Complete
3 Changes Complete
4 Changes Complete
5 Changes Complete


In [936]:
# uncomment to check it worked
# prescriber["Cred1"].value_counts().sort_values(ascending = False)

In [937]:
# MD Records:

for x in range (0, 6):
    prescriber = prescriber.replace({'Cred1': r'.MD+'}, {'Cred1': 'MD'}, regex=True)
    prescriber = prescriber.replace({'Cred1': r'.MD.+'}, {'Cred1': 'MD'}, regex=True)
    prescriber = prescriber.replace({'Cred1': r'MD+'}, {'Cred1': 'MD'}, regex=True)
    prescriber = prescriber.replace({'Cred1': r'MD.+'}, {'Cred1': 'MD'}, regex=True)
    prescriber = prescriber.replace({'Cred1': r'MD'}, {'Cred1': 'MD'}, regex=True)
    prescriber = prescriber.replace({'Cred1': r'MD.'}, {'Cred1': 'MD'}, regex=True)
    
    prescriber = prescriber.replace({'Cred1': r'MBBS'}, {'Cred1': 'MD'}, regex=True)
    prescriber = prescriber.replace({'Cred1': r'PHYSICIAN'}, {'Cred1': 'MD'}, regex=True)
    prescriber = prescriber.replace({'Cred1': r'MBBCH'}, {'Cred1': 'MD'}, regex=True)
    prescriber = prescriber.replace({'Cred1': r'MB'}, {'Cred1': 'MD'}, regex=True)
    
    print (x, 'Changes Complete')

0 Changes Complete
1 Changes Complete
2 Changes Complete
3 Changes Complete
4 Changes Complete
5 Changes Complete


In [938]:
# uncomment to check it worked
# prescriber["Cred1"].value_counts().sort_values(ascending = False)

In [939]:
# PA Records:

for x in range (0, 6):
    prescriber = prescriber.replace({'Cred1': r'.PA+'}, {'Cred1': 'PA'}, regex=True)
    prescriber = prescriber.replace({'Cred1': r'.PA.+'}, {'Cred1': 'PA'}, regex=True)
    prescriber = prescriber.replace({'Cred1': r'PA+'}, {'Cred1': 'PA'}, regex=True)
    prescriber = prescriber.replace({'Cred1': r'PA.+'}, {'Cred1': 'PA'}, regex=True)
    prescriber = prescriber.replace({'Cred1': r'PA'}, {'Cred1': 'PA'}, regex=True)
    prescriber = prescriber.replace({'Cred1': r'PA.'}, {'Cred1': 'PA'}, regex=True)
    print (x, 'Changes Complete')

0 Changes Complete
1 Changes Complete
2 Changes Complete
3 Changes Complete
4 Changes Complete
5 Changes Complete


In [940]:
# uncomment to check it worked
#prescriber["Cred1"].value_counts().sort_values(ascending = False)

In [941]:
# DDS (Dentist) Records:
prescriber = prescriber.replace({'Cred1': r'DMD'}, {'Cred1': 'DDS'}, regex=True)
prescriber = prescriber.replace({'Cred1': r'DDS'}, {'Cred1': 'DDS'}, regex=True)
prescriber = prescriber.replace({'Cred1': r'DDS.+'}, {'Cred1': 'DDS'}, regex=True)
prescriber = prescriber.replace({'Cred1': r'DENTIST'}, {'Cred1': 'DDS'}, regex=True)


# OD (Optometrist) Records:
prescriber = prescriber.replace({'Cred1': r'OPHTHALMOLGIST'}, {'Cred1': 'OD'}, regex=True)
prescriber = prescriber.replace({'Cred1': r'OPTOMETRIST'}, {'Cred1': 'OD'}, regex=True)
prescriber = prescriber.replace({'Cred1': r'FAAO'}, {'Cred1': 'OD'}, regex=True)

# fill records where the physician name was incorrectly entered in the credentials with "unknown"
prescriber = prescriber.replace({'Cred1': r'ANNA'}, {'Cred1': 'Unknown'}, regex=True)
prescriber = prescriber.replace({'Cred1': r'GREG'}, {'Cred1': 'Unknown'}, regex=True)
prescriber = prescriber.replace({'Cred1': r'HERSCHEL'}, {'Cred1': 'Unknown'}, regex=True)
prescriber = prescriber.replace({'Cred1': r'EIN-'}, {'Cred1': 'Unknown'}, regex=True)
prescriber = prescriber.replace({'Cred1': r'BRIAN'}, {'Cred1': 'Unknown'}, regex=True)
prescriber = prescriber.replace({'Cred1': r'FLORA'}, {'Cred1': 'Unknown'}, regex=True)
prescriber = prescriber.replace({'Cred1': r'GERALD'}, {'Cred1': 'Unknown'}, regex=True)
prescriber = prescriber.replace({'Cred1': r'JAMES'}, {'Cred1': 'Unknown'}, regex=True)

# Fix DPM error
prescriber = prescriber.replace({'Cred1': r'DPM`'}, {'Cred1': 'DPM'}, regex=True)


print ('Changes Complete')

Changes Complete


In [942]:
# uncomment to check it worked
#prescriber["Cred1"].value_counts().sort_values(ascending = False)

**Now let's look at the Cred1 and Cred2 columns together so we can fix places where the terminal degree wasn't in the Cred1 column.**

In [943]:
# Uncomment each ONE AT A TIME to inspect and find rows where we need to add Cred 1
#prescriber[prescriber.Cred2 == 'PA']
#prescriber[prescriber.Cred2 == 'MD']
#prescriber[prescriber.Cred2 == 'NP']
#prescriber[prescriber.Cred2 == 'DDS']
#prescriber[prescriber.Cred2 == 'OD']

**From the last step, I see that I need to change a few items**

In [944]:
prescriber = prescriber.replace({'Cred1': r'PHP'}, {'Cred1': 'MD'}, regex=True)
prescriber = prescriber.replace({'Cred1': r'FACEP'}, {'Cred1': 'MD'}, regex=True)
prescriber = prescriber.replace({'Cred1': r'^D$'}, {'Cred1': 'MD'}, regex=True)

prescriber = prescriber.replace({'Cred1': r'C-'}, {'Cred1': 'NP'}, regex=True)
prescriber = prescriber.replace({'Cred1': r'ADULT'}, {'Cred1': 'NP'}, regex=True)
prescriber = prescriber.replace({'Cred1': r'DR'}, {'Cred1': 'NP'}, regex=True)
prescriber = prescriber.replace({'Cred1': r'MSN'}, {'Cred1': 'NP'}, regex=True)

In [945]:
# uncomment to check it worked
prescriber["Cred1"].value_counts().sort_values(ascending = False)

MD         15375
NP          2469
DDS         2080
PA          1755
DO          1492
OD           550
DPM          363
MS            31
CNM           21
PHD           13
ND            13
Unknown        8
BDS            4
MHS            4
RPH            4
MEDICAL        3
N              3
FAMILY         3
DC             3
BS             2
PH             2
MED            2
MMS            2
DDA            2
PT             2
MHSA           1
MN             1
C              1
RXN            1
MHSC           1
PMH            1
PD             1
AT             1
DPT            1
PHC            1
BOARD          1
PSYD           1
MFTI           1
NFP            1
GENERAL        1
MPT            1
M              1
DOCTOR         1
OTRL           1
MSM            1
MSSW           1
BSN            1
OO             1
MT             1
ATC            1
BSHS           1
DNSC           1
MNS            1
DP             1
PCS            1
BA             1
Name: Cred1, dtype: int64

In [946]:
#I know used the below code (a few examples shown) to go through the results from the step above that had only a single record. 
#I did this to find out what was in the Cred2 column

#prescriber[prescriber.Cred1 == 'BA']
#prescriber[prescriber.Cred1 == 'PCS']
#prescriber[prescriber.Cred1 == 'MS']...

**By using the code in the cell above, I found several things to update in Cred1**

**I also looked at the results and decided to drop rows with credentials that had less than 32 records.**

In [947]:
#Making changes to Cred1 
prescriber = prescriber.replace({'Cred1': r'BA'}, {'Cred1': 'DO'}, regex=True)
prescriber = prescriber.replace({'Cred1': r'DNSC'}, {'Cred1': 'NP'}, regex=True)
prescriber = prescriber.replace({'Cred1': r'BSHS'}, {'Cred1': 'NP'}, regex=True)
prescriber = prescriber.replace({'Cred1': r'ATC'}, {'Cred1': 'NP'}, regex=True)
prescriber = prescriber.replace({'Cred1': r'OO'}, {'Cred1': 'OD'}, regex=True)
prescriber = prescriber.replace({'Cred1': r'BSN'}, {'Cred1': 'NP'}, regex=True)
prescriber = prescriber.replace({'Cred1': r'MSSW'}, {'Cred1': 'LSW'}, regex=True)
prescriber = prescriber.replace({'Cred1': r'MSM'}, {'Cred1': 'PA'}, regex=True)
prescriber = prescriber.replace({'Cred1': r'OTRL'}, {'Cred1': 'OT'}, regex=True)
prescriber = prescriber.replace({'Cred1': r'DOCTOR'}, {'Cred1': 'OD'}, regex=True)
prescriber = prescriber.replace({'Cred1': r'^M$'}, {'Cred1': 'MD'}, regex=True)
prescriber = prescriber.replace({'Cred1': r'MPT'}, {'Cred1': 'PT'}, regex=True)
prescriber = prescriber.replace({'Cred1': r'DPT'}, {'Cred1': 'PT'}, regex=True)
prescriber = prescriber.replace({'Cred1': r'GENERAL'}, {'Cred1': 'MD'}, regex=True)
prescriber = prescriber.replace({'Cred1': r'NFP'}, {'Cred1': 'NP'}, regex=True)
prescriber = prescriber.replace({'Cred1': r'MFTI'}, {'Cred1': 'NP'}, regex=True)
prescriber = prescriber.replace({'Cred1': r'BOARD'}, {'Cred1': 'NP'}, regex=True)
prescriber = prescriber.replace({'Cred1': r'PHC'}, {'Cred1': 'DDA'}, regex=True)
prescriber = prescriber.replace({'Cred1': r'AT'}, {'Cred1': 'Unknown'}, regex=True)
prescriber = prescriber.replace({'Cred1': r'PD'}, {'Cred1': 'DDS'}, regex=True)
prescriber = prescriber.replace({'Cred1': r'PMH'}, {'Cred1': 'NP'}, regex=True)
prescriber = prescriber.replace({'Cred1': r'MN'}, {'Cred1': 'NP'}, regex=True)
prescriber = prescriber.replace({'Cred1': r'RXN'}, {'Cred1': 'NP'}, regex=True)
prescriber = prescriber.replace({'Cred1': r'^C$'}, {'Cred1': 'NP'}, regex=True)
prescriber = prescriber.replace({'Cred1': r'MN'}, {'Cred1': 'NP'}, regex=True)
prescriber = prescriber.replace({'Cred1': r'MHSA'}, {'Cred1': 'PA'}, regex=True)
prescriber = prescriber.replace({'Cred1': r'MMS'}, {'Cred1': 'PA'}, regex=True)
prescriber = prescriber.drop(prescriber.index[6516])
prescriber = prescriber.replace({'Cred1': r'BS'}, {'Cred1': 'PA'}, regex=True)
prescriber = prescriber[prescriber.Cred1 != 'MED']
prescriber = prescriber[prescriber.Cred1 != 'DC']
prescriber = prescriber.replace({'Cred1': r'FAMILY'}, {'Cred1': 'NP'}, regex=True)
prescriber = prescriber.replace({'Cred1': r'^N$'}, {'Cred1': 'NP'}, regex=True)
prescriber = prescriber.replace({'Cred1': r'MEDICAL'}, {'Cred1': 'MD'}, regex=True)
prescriber = prescriber.replace({'Cred1': r'RPH'}, {'Cred1': 'DDA'}, regex=True)
prescriber = prescriber.replace({'Cred1': r'MHS'}, {'Cred1': 'PA'}, regex=True)
prescriber = prescriber.replace({'Cred1': r'BDS'}, {'Cred1': 'DDS'}, regex=True)



In [948]:
# uncomment to check it worked
prescriber["Cred1"].value_counts().sort_values(ascending = False)

MD         15380
NP          2486
DDS         2085
PA          1764
DO          1493
OD           552
DPM          363
MS            31
CNM           21
ND            13
PHD           13
Unknown        9
DDA            7
PT             4
PH             2
PCS            1
PSYD           1
PAC            1
MT             1
NPS            1
OT             1
LSW            1
DP             1
Name: Cred1, dtype: int64

In [949]:

# dropping rows that had a credential that appeared less than 32 times.
prescriber = prescriber[prescriber.Cred1 != 'DP']
prescriber = prescriber[prescriber.Cred1 != 'MT']
prescriber = prescriber[prescriber.Cred1 != 'LSW']
prescriber = prescriber[prescriber.Cred1 != 'NPS']
prescriber = prescriber[prescriber.Cred1 != 'PSYD']
prescriber = prescriber[prescriber.Cred1 != 'PCS']
prescriber = prescriber[prescriber.Cred1 != 'OT']
prescriber = prescriber[prescriber.Cred1 != 'PH']
prescriber = prescriber[prescriber.Cred1 != 'PT']
prescriber = prescriber[prescriber.Cred1 != 'DDA']
prescriber = prescriber[prescriber.Cred1 != 'Unknown']
prescriber = prescriber[prescriber.Cred1 != 'MS']
prescriber = prescriber[prescriber.Cred1 != 'CNM']
prescriber = prescriber[prescriber.Cred1 != 'ND']
prescriber = prescriber[prescriber.Cred1 != 'PHD']

**We can now look at our Cred1 column and see that our data looks MUCH more clean than what we started with!**

In [950]:
# uncomment to check it worked
prescriber["Cred1"].value_counts().sort_values(ascending = False)

MD     15380
NP      2486
DDS     2085
PA      1764
DO      1493
OD       552
DPM      363
PAC        1
Name: Cred1, dtype: int64

**Now, we'll need to drop the "NPI" column since this is a different/individual id number for each prescriber.**

**We will also drop the "Cred2" feature and change the name of "Cred1" to "Degree"**

**We will also change the name of Gender to DR_Gender to clarify that this is the gender of the PRESCRIBER and not the patient**

In [951]:
prescriber.head()

,NPI,Gender,State,Credentials,Specialty,ABILIFY,ACETAMINOPHEN.CODEINE,ACYCLOVIR,ADVAIR.DISKUS,AGGRENOX,...,VOLTAREN,VYTORIN,WARFARIN.SODIUM,XARELTO,ZETIA,ZIPRASIDONE.HCL,ZOLPIDEM.TARTRATE,Opioid.Prescriber,Cred1,Cred2
0,1710982582,M,TX,DDS,Dentist,0,0,0,0,0,...,0,0,0,0,0,0,0,1,DDS,None
1,1245278100,F,AL,MD,General Surgery,0,0,0,0,0,...,0,0,0,0,0,0,35,1,MD,None
2,1427182161,F,NY,MD,General Practice,0,0,0,0,0,...,0,0,0,0,0,0,25,0,MD,None
3,1669567541,M,AZ,MD,Internal Medicine,0,43,0,0,0,...,0,0,0,0,0,0,0,1,MD,None
4,1679650949,M,NV,MD,Hematology/Oncology,0,0,0,0,0,...,0,0,17,28,0,0,0,1,MD,None


In [952]:
prescriber = prescriber.drop('NPI', axis=1)
prescriber = prescriber.drop('Cred2', axis=1)
prescriber = prescriber.drop('Credentials', axis=1)
prescriber.rename(columns={'Cred1': 'Degree', 'Gender': 'DR_Gender'}, inplace=True)
prescriber.head()

,DR_Gender,State,Specialty,ABILIFY,ACETAMINOPHEN.CODEINE,ACYCLOVIR,ADVAIR.DISKUS,AGGRENOX,ALENDRONATE.SODIUM,ALLOPURINOL,...,VESICARE,VOLTAREN,VYTORIN,WARFARIN.SODIUM,XARELTO,ZETIA,ZIPRASIDONE.HCL,ZOLPIDEM.TARTRATE,Opioid.Prescriber,Degree
0,M,TX,Dentist,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,DDS
1,F,AL,General Surgery,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,35,1,MD
2,F,NY,General Practice,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,25,0,MD
3,M,AZ,Internal Medicine,0,43,0,0,0,21,0,...,0,0,0,0,0,0,0,0,1,MD
4,M,NV,Hematology/Oncology,0,0,0,0,0,0,0,...,0,0,0,17,28,0,0,0,1,MD


**Great!! Now we've done a lot. We have all of the non-drug columns cleaned up pretty well.**

**Now, we need to address the drug columns in PART B.**

## **PART 2:**

## **Wrangling and EDA of drug features.**

In [18]:
# I'll now deal with the last problem child ...
em_prescriber = em_prescriber.replace({'Credentials': r'BA,,DC,DOJD,LLM'}, {'Credentials': 'DO,JD,DC,LLM,BA'}, regex=True) # add extra comman after DO due to first command above

In [74]:
em_prescriber["Credentials"].value_counts().sort_values(ascending = False)

NameError: name 'em_prescriber' is not defined

**Now, let's try to split the data into a "Title" and "OtherDegrees" features**

In [20]:
# new data frame with split value columns 
new = em_prescriber["Credentials"].str.split(",", n = 5, expand = True) 

# making seperate title column from new data frame 
em_prescriber['Title']= new[0] 
  
# making seperate OtherDegree1 from new data frame 
em_prescriber["OtherDegree1"]= new[1] 

# making seperate OtherDegree(n)columns from new data frame 
# NOTE!! IT IS ASSUMED THAT TO HAVE AN MD OR DO DEGREE THAT YOU WOULD HAVE A BA OR BS DEGREE ALREADY. SO, I'M NOT INTERESTED IN THOSE.
#       THERE ARE ALSO ONLY 2 RECORDS WITH DEGREES/CERTIFICATES BEYOND THE 1ST 2 INDICES OF THE STRINGS ABOVE.
# THEREFORE, I AM ONLY INTERESTED IN THE FIRST 2 LEVELS BEYOND THE TERMINAL DEGREE...
em_prescriber["OtherDegree2"]= new[2] 
em_prescriber["OtherDegree3"]= new[3] 

# Dropping old Credentials column 
em_prescriber.drop(columns =["Credentials"], inplace = True) 

In [21]:
em_prescriber.head()

,NPI,Gender,State,Specialty,ABILIFY,ACETAMINOPHEN.CODEINE,ACYCLOVIR,ADVAIR.DISKUS,AGGRENOX,ALENDRONATE.SODIUM,...,WARFARIN.SODIUM,XARELTO,ZETIA,ZIPRASIDONE.HCL,ZOLPIDEM.TARTRATE,Opioid.Prescriber,Title,OtherDegree1,OtherDegree2,OtherDegree3
68,1114977758,M,NE,Emergency Medicine,0,0,0,0,0,0,...,0,0,0,0,0,1,MD,None,None,None
91,1265477343,M,TX,Emergency Medicine,0,0,0,0,0,0,...,0,0,0,0,0,1,DO,None,None,None
108,1588612477,F,CA,Emergency Medicine,0,0,0,0,0,0,...,0,0,0,0,0,1,Unknown,None,None,None
109,1881825461,M,FL,Emergency Medicine,0,0,0,0,0,0,...,0,0,0,0,0,1,DO,None,None,None
118,1467627729,M,FL,Emergency Medicine,0,0,0,0,0,0,...,0,0,0,0,0,1,Unknown,None,None,None


In [22]:
em_prescriber["Title"].value_counts().sort_values(ascending = False)

MD         884
DO         169
Unknown     34
Name: Title, dtype: int64

In [23]:
em_prescriber["OtherDegree1"].value_counts().sort_values(ascending = False)

MS       4
PHD      4
FACEP    3
MBA      3
MPH      1
JD       1
Name: OtherDegree1, dtype: int64

In [24]:
em_prescriber["OtherDegree2"].value_counts().sort_values(ascending = False)

BS       1
DC       1
FAAEM    1
Name: OtherDegree2, dtype: int64

In [25]:
em_prescriber["OtherDegree3"].value_counts().sort_values(ascending = False)

LLM    1
Name: OtherDegree3, dtype: int64

Since those with degrees in the "OtherDegree" lists 2 and 3 also have degrees in the 1st lis, I will drop the "OtherDeree2 and 3" columns 

I will also now turn the strings in the "Title" and "OtherDegree" columns to integers

In [26]:
# drop columns
em_prescriber.drop(columns =["OtherDegree2", "OtherDegree3"], inplace = True) 

In [27]:
em_prescriber.head()

,NPI,Gender,State,Specialty,ABILIFY,ACETAMINOPHEN.CODEINE,ACYCLOVIR,ADVAIR.DISKUS,AGGRENOX,ALENDRONATE.SODIUM,...,VOLTAREN,VYTORIN,WARFARIN.SODIUM,XARELTO,ZETIA,ZIPRASIDONE.HCL,ZOLPIDEM.TARTRATE,Opioid.Prescriber,Title,OtherDegree1
68,1114977758,M,NE,Emergency Medicine,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,MD,None
91,1265477343,M,TX,Emergency Medicine,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,DO,None
108,1588612477,F,CA,Emergency Medicine,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,Unknown,None
109,1881825461,M,FL,Emergency Medicine,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,DO,None
118,1467627729,M,FL,Emergency Medicine,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,Unknown,None


In [28]:
em_prescriber["OtherDegree1"].value_counts().sort_values(ascending = False)

MS       4
PHD      4
FACEP    3
MBA      3
MPH      1
JD       1
Name: OtherDegree1, dtype: int64

In [29]:
em_prescriber['Title'] = em_prescriber['Title'].replace({'MD': 0, 'DO': 1, 'Unknown': 2 })

In [30]:
em_prescriber["OtherDegree1"].value_counts().sort_values(ascending = False)

MS       4
PHD      4
FACEP    3
MBA      3
MPH      1
JD       1
Name: OtherDegree1, dtype: int64

In [31]:
em_prescriber['OtherDegree1'] = em_prescriber['OtherDegree1'].replace({'PHD':1, 'MS':1, 'FACEP':1, 'MBA':1, 'MPH':1, 'JD': 1})

In [32]:
em_prescriber['OtherDegree1'] = em_prescriber['OtherDegree1'].fillna(0).astype(int)

In [33]:
em_prescriber.head()

,NPI,Gender,State,Specialty,ABILIFY,ACETAMINOPHEN.CODEINE,ACYCLOVIR,ADVAIR.DISKUS,AGGRENOX,ALENDRONATE.SODIUM,...,VOLTAREN,VYTORIN,WARFARIN.SODIUM,XARELTO,ZETIA,ZIPRASIDONE.HCL,ZOLPIDEM.TARTRATE,Opioid.Prescriber,Title,OtherDegree1
68,1114977758,M,NE,Emergency Medicine,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
91,1265477343,M,TX,Emergency Medicine,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
108,1588612477,F,CA,Emergency Medicine,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,2,0
109,1881825461,M,FL,Emergency Medicine,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
118,1467627729,M,FL,Emergency Medicine,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,2,0


In [34]:
em_prescriber["OtherDegree1"].value_counts().sort_values(ascending = False)

0    1071
1      16
Name: OtherDegree1, dtype: int64

**Now, let's look at one of the drug columns and see what the data looks like**

I'll look at the Acyclovir inititally

In [35]:
em_prescriber["AGGRENOX"].value_counts().sort_values(ascending = False)

0     1081
11       1
12       1
13       1
17       1
27       1
69       1
Name: AGGRENOX, dtype: int64

We can see that they are integers and thus would be fine the way they are for downstream machine learning algorithms

These are number of prescriptions by that specific prescriber for the time in which the data was collected (2014), so this makes sense.

**We can see that the NPI number is unique for each prescriber. Therefore, we will drop it from the dataset**

In [36]:
em_prescriber = em_prescriber.drop(['NPI'], axis = 1)

In [37]:
em_prescriber.head()

,Gender,State,Specialty,ABILIFY,ACETAMINOPHEN.CODEINE,ACYCLOVIR,ADVAIR.DISKUS,AGGRENOX,ALENDRONATE.SODIUM,ALLOPURINOL,...,VOLTAREN,VYTORIN,WARFARIN.SODIUM,XARELTO,ZETIA,ZIPRASIDONE.HCL,ZOLPIDEM.TARTRATE,Opioid.Prescriber,Title,OtherDegree1
68,M,NE,Emergency Medicine,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
91,M,TX,Emergency Medicine,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
108,F,CA,Emergency Medicine,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,2,0
109,M,FL,Emergency Medicine,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
118,M,FL,Emergency Medicine,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,2,0


**Let's change the Gender values from categorical to ordinal now**

In [38]:
em_prescriber['Gender'] = em_prescriber['Gender'].replace({'M':0, 'F':1 })

**Finally, let's drop the Specialty feature now.**

In [39]:
em_prescriber = em_prescriber.drop(['Specialty'], axis = 1)

In [40]:
em_prescriber.head()

,Gender,State,ABILIFY,ACETAMINOPHEN.CODEINE,ACYCLOVIR,ADVAIR.DISKUS,AGGRENOX,ALENDRONATE.SODIUM,ALLOPURINOL,ALPRAZOLAM,...,VOLTAREN,VYTORIN,WARFARIN.SODIUM,XARELTO,ZETIA,ZIPRASIDONE.HCL,ZOLPIDEM.TARTRATE,Opioid.Prescriber,Title,OtherDegree1
68,0,NE,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
91,0,TX,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
108,1,CA,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,2,0
109,0,FL,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
118,0,FL,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,2,0


In [41]:
# We know look at the column info and find the column "OtherDegree1" is int32, whereas everything else is int64

# em_prescriber.info(all)

In [42]:
em_prescriber['OtherDegree1'] = em_prescriber.OtherDegree1.astype(np.int64)

In [43]:
# em_prescriber.info(all)

**So, now all the data is int64 except the State feature. I'd rather not change it yet, to make initial data visualization easier, but let's look at it...**

In [44]:
em_prescriber['State'].value_counts().sort_values(ascending=False)

CA    113
TX     73
FL     67
NY     62
PA     52
MI     47
IL     47
OH     44
IN     36
GA     34
VA     31
NC     29
MA     27
MO     25
NJ     25
WA     24
CO     23
AZ     21
MD     19
KY     19
SC     18
AL     17
MN     17
TN     16
WI     16
NV     15
CT     15
WV     15
LA     14
OR     13
OK     13
MS     12
UT     11
IA     11
NM      9
AR      6
ND      6
RI      5
DC      5
PR      5
DE      5
HI      4
MT      4
KS      4
ME      3
NE      3
ID      2
VT      2
SD      1
NH      1
VI      1
Name: State, dtype: int64

## C. Data Visualizations and Analysis

Let's start by looking for any correlations...

In [45]:
em_prescriber.head()

,Gender,State,ABILIFY,ACETAMINOPHEN.CODEINE,ACYCLOVIR,ADVAIR.DISKUS,AGGRENOX,ALENDRONATE.SODIUM,ALLOPURINOL,ALPRAZOLAM,...,VOLTAREN,VYTORIN,WARFARIN.SODIUM,XARELTO,ZETIA,ZIPRASIDONE.HCL,ZOLPIDEM.TARTRATE,Opioid.Prescriber,Title,OtherDegree1
68,0,NE,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
91,0,TX,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
108,1,CA,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,2,0
109,0,FL,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
118,0,FL,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,2,0


In [46]:
# Correlation between Title and Opioid.Prescriber
em_prescriber[["Title", "Opioid.Prescriber"]].groupby(["Title"], as_index=False).mean().sort_values(by="Opioid.Prescriber", ascending=False)

,Title,Opioid.Prescriber
0,0,0.964932
1,1,0.952663
2,2,0.911765


In [61]:
# Correlation between State and Opioid.Prescriber
state_corr = em_prescriber[["State", "Opioid.Prescriber"]].groupby(["State"], as_index=False).mean().sort_values(by="Opioid.Prescriber", ascending=False)
state_corr

,State,Opioid.Prescriber
0,AL,1.000000
20,ME,1.000000
1,AR,1.000000
26,NC,1.000000
27,ND,1.000000
28,NE,1.000000
29,NH,1.000000
31,NM,1.000000
32,NV,1.000000
39,RI,1.000000


In [68]:
# Correlation between Acetaminophen/codeine and Opioid.Prescriber
acetcod_corr = em_prescriber[["ACTIQ", "Opioid.Prescriber"]].groupby(["ACTIQ"], as_index=False).mean().sort_values(by="Opioid.Prescriber", ascending=False)
acetcod_corr

KeyError: "['ACTIQ'] not in index"

## D. Split the data into test and training sets before we start data visualizations and analysis

Before we start using data visualization to look at the data closer, let's split it into a train and test set and then only look at the train set.

Our target is "Opioid.Prescriber" since we are trying to predict which prescribers might be considered opioid prescribers in the emergency medicne department.

In [ ]:
# Create target object and call it y
y=em_prescriber['Opioid.Prescriber']

# Create X
features = em_prescriber.columns.drop('Opioid.Prescriber')
X = em_prescriber[features]

# Split into validation and training data
train_X, val_X, train_y, val_y = train_test_split(X, y, test_size = 0.2, random_state=1)

**Now let's see the information about the dataset...**

In [ ]:
prescriber.info()

#### 2. Opioid Dataset

In [67]:
opioids.head()

,Drug Name,Generic Name
0,ABSTRAL,FENTANYL CITRATE
1,ACETAMINOPHEN-CODEINE,ACETAMINOPHEN WITH CODEINE
2,ACTIQ,FENTANYL CITRATE
3,ASCOMP WITH CODEINE,CODEINE/BUTALBITAL/ASA/CAFFEIN
4,ASPIRIN-CAFFEINE-DIHYDROCODEIN,DIHYDROCODEINE/ASPIRIN/CAFFEIN


In [ ]:
opioids.tail()

In [69]:
opioids.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113 entries, 0 to 112
Data columns (total 2 columns):
Drug Name       113 non-null object
Generic Name    113 non-null object
dtypes: object(2)
memory usage: 1.8+ KB


#### 3. Overdose Dataset

In [ ]:
overdose.head()

In [ ]:
overdose.tail()

In [ ]:
overdose.info()